In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import warnings
import time

In [2]:
images_paths = list()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        image_path = os.path.join(dirname, filename)
        if image_path.endswith('.jpg') or image_path.endswith(".png"):
            images_paths.append(image_path)

In [3]:
class ColorizationDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths

    def __getitem__(self, idx):
        return self.paths[idx]

    def __len__(self):
        return len(self.paths)
    
    
def make_dataloader(batch_size=16, n_workers=2, pin_memory=True, **kwargs):
    dataset = ColorizationDataset(**kwargs)
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=n_workers,
                            pin_memory=pin_memory)
    return dataloader

In [4]:
np.random.seed(123)
all_images = images_paths
train_range = int(0.8 * len(all_images))
rand_idxs = np.random.permutation(len(all_images))
train_idxs = rand_idxs[:train_range] 
val_idxs = rand_idxs[train_range:] 
train_paths = [all_images[x] for x in train_idxs]
val_paths = [all_images[x] for x in val_idxs]
print(len(train_paths), len(val_paths))

11019 2755


In [5]:
train_loader = make_dataloader(paths=train_paths)
val_loader = make_dataloader(paths=val_paths)
len(train_loader), len(val_loader)

(689, 173)

In [6]:
def get_name_from_path(path):
    return path.split('/')[-1]

In [7]:
def save_grayscale_batch(batch):
    for img_path in batch:
        img_rgb = Image.open(img_path)
        img_gray = img_rgb.convert('L')
        img_gray.save(f'{get_name_from_path(img_path)}')

In [8]:
def save_batch(batch):
    for img_path in batch:
        img_rgb = Image.open(img_path)
        img_rgb.save(f'{get_name_from_path(img_path)}')

In [9]:
def load_grayscale_batch(batch):
    return [f'{get_name_from_path(img_path)}' for img_path in batch]

In [10]:
def del_grayscale_batch(batch):
    for img_path in batch:
        os.remove(f'{get_name_from_path(img_path)}')

In [11]:
import torch
from torch import nn

class BaseColor(nn.Module):
    def __init__(self):
        super(BaseColor, self).__init__()

        self.l_cent = 50.
        self.l_norm = 100.
        self.ab_norm = 110.

    def normalize_l(self, in_l):
        return (in_l-self.l_cent)/self.l_norm

    def unnormalize_l(self, in_l):
        return in_l*self.l_norm + self.l_cent

    def normalize_ab(self, in_ab):
        return in_ab/self.ab_norm

    def unnormalize_ab(self, in_ab):
        return in_ab*self.ab_norm



class ECCVGenerator(BaseColor):
    def __init__(self, norm_layer=nn.BatchNorm2d):
        super(ECCVGenerator, self).__init__()

        model1=[nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[norm_layer(64),]

        model2=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[norm_layer(128),]

        model3=[nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[norm_layer(256),]

        model4=[nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[norm_layer(512),]

        model5=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[norm_layer(512),]

        model6=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[norm_layer(512),]

        model7=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[norm_layer(512),]

        model8=[nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]

        model8+=[nn.Conv2d(256, 313, kernel_size=1, stride=1, padding=0, bias=True),]

        self.model1 = nn.Sequential(*model1)
        self.model2 = nn.Sequential(*model2)
        self.model3 = nn.Sequential(*model3)
        self.model4 = nn.Sequential(*model4)
        self.model5 = nn.Sequential(*model5)
        self.model6 = nn.Sequential(*model6)
        self.model7 = nn.Sequential(*model7)
        self.model8 = nn.Sequential(*model8)

        self.softmax = nn.Softmax(dim=1)
        self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)
        self.upsample4 = nn.Upsample(scale_factor=4, mode='bilinear')

    def forward(self, input_l):
        conv1_2 = self.model1(self.normalize_l(input_l))
        conv2_2 = self.model2(conv1_2)
        conv3_3 = self.model3(conv2_2)
        conv4_3 = self.model4(conv3_3)
        conv5_3 = self.model5(conv4_3)
        conv6_3 = self.model6(conv5_3)
        conv7_3 = self.model7(conv6_3)
        conv8_3 = self.model8(conv7_3)
        out_reg = self.model_out(self.softmax(conv8_3))

        return self.unnormalize_ab(self.upsample4(out_reg))

def eccv16(pretrained=True):
    model = ECCVGenerator()
    if(pretrained):
        import torch.utils.model_zoo as model_zoo
        model.load_state_dict(model_zoo.load_url('https://colorizers.s3.us-east-2.amazonaws.com/colorization_release_v2-9b330a0b.pth',map_location='cpu',check_hash=True))
    return model


from PIL import Image
import numpy as np
from skimage import color
import torch
import torch.nn.functional as F

# def load_img(img_path):
def load_imgs(img_path_list):
    out_np_list = [np.asarray(Image.open(img_path).convert('RGB'))
                   for img_path in img_path_list]
    
    # if(out_np.ndim==2):
    #    out_np = np.tile(out_np[:,:,None],3)
    for i, out_np in enumerate(out_np_list):
        if out_np.ndim==2:
            out_np_list[i] = np.tile(out_np[:,:,None],3)
    # return out_np
    return out_np_list

def resize_img(img, HW=(256,256), resample=3):
    return np.asarray(Image.fromarray(img).resize((HW[1],HW[0]), resample=resample))

# def preprocess_img(img_rgb_orig, HW=(256,256), resample=3):
def preprocess_imgs(img_rgb_orig_list, HW=(256,256), resample=3):
    # return original size L and resized L as torch Tensors
    # img_rgb_rs = resize_img(img_rgb_orig, HW=HW, resample=resample)
    img_rgb_rs_list = [resize_img(img_rgb_orig, HW=HW, resample=resample)
                       for img_rgb_orig in img_rgb_orig_list]

    # img_lab_orig = color.rgb2lab(img_rgb_orig)
    # img_lab_rs = color.rgb2lab(img_rgb_rs)
    img_lab_orig_list = [color.rgb2lab(img_rgb_orig)
                         for img_rgb_orig in img_rgb_orig_list]
    img_lab_rs_list = [color.rgb2lab(img_rgb_rs)
                       for img_rgb_rs in img_rgb_rs_list]

    # img_l_orig = img_lab_orig[:,:,0]
    # img_l_rs = img_lab_rs[:,:,0]
    img_l_orig_list = [img_lab_orig[:,:,0]
                       for img_lab_orig in img_lab_orig_list]
    img_l_rs_list = [img_lab_rs[:,:,0]
                     for img_lab_rs in img_lab_rs_list]
    
    # FOR GT
    # (256, 256, 2) 1 2 -> 0 1
    img_ab_rs_list = [torch.transpose(torch.transpose(torch.Tensor(img_lab_rs[:,:,1:3]), 1, 2), 0, 1)
                     for img_lab_rs in img_lab_rs_list]

    # RESHAPE FIRST
    img_l_rs_list_reshaped = [torch.Tensor(img_l_rs)[None, None, :, :]
                              for img_l_rs in img_l_rs_list]
    img_ab_rs_list_reshaped = [torch.Tensor(img_ab_rs)[None,:, :, :]
                              for img_ab_rs in img_ab_rs_list]

    # tens_orig_l = torch.Tensor(img_l_orig)[None,None,:,:]
    # tens_rs_l = torch.Tensor(img_l_rs)[None,None,:,:]
    tens_orig_l_list = [torch.Tensor(img_l_orig)[None,None,:,:]
                        for img_l_orig in img_l_orig_list]
    
    batch_size = len(img_rgb_orig_list)
    tens_rs_l_tensor = torch.cat(img_l_rs_list_reshaped, dim=0)
    tens_rs_ab_tensor = torch.cat(img_ab_rs_list_reshaped, dim=0)

    return (tens_orig_l_list, tens_rs_l_tensor, tens_rs_ab_tensor)

# def postprocess_tens(tens_orig_l, out_ab, mode='bilinear'):
def postprocess_imgs(tens_orig_l_list, out_ab, mode='bilinear'):
    # tens_orig_l     1 x 1 x H_orig x W_orig
    # out_ab         1 x 2 x H x W
    
    # tens_orig_l_list: (16 elements) 1 x 1 x H_orig x W_orig
    # out_ab                          16 x 2 x H x W

    # HW_orig = tens_orig_l.shape[2:]
    # HW = out_ab.shape[2:]
    HW_orig_list = [tens_orig_l.shape[2:] for tens_orig_l in tens_orig_l_list]  # 16 ele H_orig x W_orig
    HW = out_ab.shape[2:]  # (H, W)

    # call resize function if needed
    #if(HW_orig[0]!=HW[0] or HW_orig[1]!=HW[1]):
    #    out_ab_orig = F.interpolate(out_ab, size=HW_orig, mode='bilinear')
    #else:
    #    out_ab_orig = out_ab
    
    out_ab_orig_list = list()
    for i, HW_orig in enumerate(HW_orig_list):
        if(HW_orig[0]!=HW[0] or HW_orig[1]!=HW[1]):
            out_ab_orig = F.interpolate(out_ab[i][None, :, :, :], size=HW_orig, mode='bilinear')
        else:
            out_ab_orig = out_ab[i][None, :, :, :]     
        out_ab_orig_list.append(out_ab_orig)

    # out_lab_orig = torch.cat((tens_orig_l, out_ab_orig), dim=1)
    out_lab_orig_list = list()
    for i in range(len(out_ab_orig_list)):
        out_lab_orig_list.append(
            torch.cat((tens_orig_l_list[i].cuda(), out_ab_orig_list[i]), dim=1)
        )
    
    # return color.lab2rgb(out_lab_orig.data.cpu().numpy()[0,...].transpose((1,2,0)))
    return [color.lab2rgb(out_lab_orig.data.cpu().numpy()[0,...].transpose((1,2,0)))
           for out_lab_orig in out_lab_orig_list]

In [12]:
model = eccv16(pretrained=True).cuda()
model.train()
# model.load_state_dict(torch.load('/kaggle/input/eccv16-train-epoch-14/model_14.pth'))
optimizer = torch.optim.AdamW(model.parameters())

Downloading: "https://colorizers.s3.us-east-2.amazonaws.com/colorization_release_v2-9b330a0b.pth" to /root/.cache/torch/hub/checkpoints/colorization_release_v2-9b330a0b.pth
100%|██████████| 123M/123M [00:04<00:00, 29.4MB/s]


(689, 173) batches

In [13]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    for epoch in range(0, 10):
        print(f'Epoch {epoch}')
        for batch_idx, batch in enumerate(train_loader):
            optimizer.zero_grad()

            original_imgs = load_imgs(batch)
            orig_l_list, rs_l_tensor, gt = preprocess_imgs(original_imgs)

            output_tensor = model(rs_l_tensor.cuda())        

            loss = F.mse_loss(output_tensor, gt.cuda())
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print(f'Batch {batch_idx}: loss={loss}')

            torch.cuda.empty_cache()

        torch.save(model.state_dict(), f'model_{epoch}')

Epoch 0
Batch 0: loss=230.8716583251953
Batch 10: loss=84.85639953613281
Batch 20: loss=93.0570297241211
Batch 30: loss=105.48080444335938
Batch 40: loss=110.83549499511719
Batch 50: loss=81.59423828125
Batch 60: loss=133.99737548828125
Batch 70: loss=93.16462707519531
Batch 80: loss=147.85980224609375
Batch 90: loss=108.5198745727539
Batch 100: loss=90.40885162353516
Batch 110: loss=75.20344543457031
Batch 120: loss=86.13192749023438
Batch 130: loss=74.51362609863281
Batch 140: loss=100.82162475585938
Batch 150: loss=168.13232421875
Batch 160: loss=66.36837768554688
Batch 170: loss=124.9740982055664
Batch 180: loss=70.95185089111328
Batch 190: loss=163.56787109375
Batch 200: loss=137.35235595703125
Batch 210: loss=108.93681335449219
Batch 220: loss=117.53683471679688
Batch 230: loss=95.5561294555664
Batch 240: loss=131.8926544189453
Batch 250: loss=53.789710998535156
Batch 260: loss=80.94195556640625
Batch 270: loss=91.37062072753906
Batch 280: loss=94.54707336425781
Batch 290: loss=9